<a href="https://colab.research.google.com/github/malinphy/Embedding_calls/blob/main/QAS_BERT_Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
import numpy as np 
import pandas as pd 
import os 
import sys 
import json
from platform import python_version

import tensorflow as tf 
from tensorflow import keras 
from tensorflow.keras import layers
from tensorflow.keras.layers import Embedding,Dense,MultiHeadAttention

import tensorflow_hub as hub
!pip install -q tensorflow-text

import tensorflow_text as text

In [49]:
print('Python Version : ' ,python_version())
print('tensorflow version : ', tf.__version__)
print('pandas version : ', pd.__version__)
print('numpy version : ', np.__version__)
print('tf_hub version : ', hub.__version__)


Python Version :  3.7.10
tensorflow version :  2.4.1
pandas version :  1.1.5
numpy version :  1.19.5
tf_hub version :  0.11.0


In [50]:
train_path = keras.utils.get_file("train.json", "https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v1.1.json")
eval_path = keras.utils.get_file("eval.json", "https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v1.1.json")
with open(train_path) as f: raw_train_data = json.load(f)
with open(eval_path) as f: raw_eval_data = json.load(f)
# max_seq_length = 384

In [51]:
print(raw_train_data.keys())
print(raw_train_data['data'][0].keys())
print(raw_train_data['data'][0]['title'][0:])
print(raw_train_data['data'][1]['title'][0:])

dict_keys(['data', 'version'])
dict_keys(['title', 'paragraphs'])
University_of_Notre_Dame
Beyoncé


In [52]:
print(raw_train_data['data'][3]['paragraphs'][0].keys())

dict_keys(['context', 'qas'])


In [53]:
print(raw_train_data['data'][0]['paragraphs'][0]['qas'])

[{'answers': [{'answer_start': 515, 'text': 'Saint Bernadette Soubirous'}], 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?', 'id': '5733be284776f41900661182'}, {'answers': [{'answer_start': 188, 'text': 'a copper statue of Christ'}], 'question': 'What is in front of the Notre Dame Main Building?', 'id': '5733be284776f4190066117f'}, {'answers': [{'answer_start': 279, 'text': 'the Main Building'}], 'question': 'The Basilica of the Sacred heart at Notre Dame is beside to which structure?', 'id': '5733be284776f41900661180'}, {'answers': [{'answer_start': 381, 'text': 'a Marian place of prayer and reflection'}], 'question': 'What is the Grotto at Notre Dame?', 'id': '5733be284776f41900661181'}, {'answers': [{'answer_start': 92, 'text': 'a golden statue of the Virgin Mary'}], 'question': 'What sits on top of the Main Building at Notre Dame?', 'id': '5733be284776f4190066117e'}]


In [54]:
print(raw_train_data['data'][0]['title'])
print(raw_train_data.keys())
print(raw_train_data['data'][0].keys())
print(raw_train_data['data'][0]['paragraphs'][0]['qas'][1]['answers'][0]['text'])
x = (raw_train_data['data'][0]['paragraphs'][0]['context'])
x

University_of_Notre_Dame
dict_keys(['data', 'version'])
dict_keys(['title', 'paragraphs'])
a copper statue of Christ


'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.'

In [55]:
raw_train_data['data'][0]['paragraphs'][0]['qas'][0]['answers'][0]['answer_start']

515

In [56]:
bert_preprocess = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'

bert_model = 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1'

bert_preprocess_layer = hub.load(bert_preprocess)
bert_layer = hub.KerasLayer(bert_model,trainable=True)

In [57]:
def fun(x):
  return (bert_preprocess_layer.tokenize([str(x)]))

In [58]:
den = ['hi']
fun(den)

<tf.RaggedTensor [[[1031], [1005], [7632], [1005], [1033]]]>

In [72]:
def text_maker (data):
  squad_examples = []
  full_context = []
  full_question = []
  full_answer = []
  title = []
  starting_index = []
  context_question =[]
  new_line =[]
  ending_index = []

  for item in data["data"]:
   
      title.append(item['title'])
      for para in item["paragraphs"]:
          context = para["context"]
          # full_context.append(context)

          

          for qa in para["qas"]:
              question = qa["question"]
              x = np.hstack((context,question))
              new_line.append(x)

              

              if "answers" in qa:
                  answer_text = qa["answers"][0]["text"]
                  full_answer.append(answer_text)
                  all_answers = [_["text"] for _ in qa["answers"]]
                  start_char_idx = qa["answers"][0]["answer_start"]
                  end_char_idx = start_char_idx + len(answer_text)
                  # print(start_char_idx)
                  starting_index.append(start_char_idx)
                  ending_index.append(end_char_idx)
                # squad_eg = Sample(question, context, start_char_idx, answer_text, all_answers)
            # else:
                # squad_eg = Sample(question, context)
            # squad_eg.preprocess()
            # squad_examples.append(squad_eg)
  x  = pd.DataFrame(new_line)
  x['start_index'] = starting_index
  x['end_index'] = ending_index
  return (x)

In [73]:
col1 = ['I live in london','weather is nice']
col2 = ['I am old.', 'my name is john']

In [75]:
all_data= text_maker(raw_train_data)
all_data

,0,1,start_index,end_index
0,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,515,541
1,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,188,213
2,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,279,296
3,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,381,420
4,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,92,126
...,...,...,...,...
87594,"Kathmandu Metropolitan City (KMC), in order to...",In what US state did Kathmandu first establish...,229,235
87595,"Kathmandu Metropolitan City (KMC), in order to...",What was Yangon previously known as?,414,421
87596,"Kathmandu Metropolitan City (KMC), in order to...",With what Belorussian city does Kathmandu have...,476,481
87597,"Kathmandu Metropolitan City (KMC), in order to...",In what year did Kathmandu create its initial ...,199,203


In [ ]:

# train_context, train_question,train_answer,train_answer_index,train_titles,con_ques= text_maker(raw_train_data)
# test_context, test_question, test_answer,test_answer_index,test_titles = text_maker(raw_eval_data)

In [ ]:
# text_test = ['this is such an amazing movie!']

# sample_tokenizer = bert_preprocess_layer(text_test)
# print(f'Keys       : {list(sample_tokenizer.keys())}')
# print(f'Shape      : {sample_tokenizer["input_word_ids"].shape}')
# print(f'Word Ids   : {sample_tokenizer["input_word_ids"][0, :12]}')
# print(f'Input Mask : {sample_tokenizer["input_mask"][0, :12]}')
# print(f'Type Ids   : {sample_tokenizer["input_type_ids"][0, :12]}')

### BELOW THIS LINE INPUT_IDS CAN BE CONCAT
### a = bert_preprocess_layer.tokenize([train_context[0]])
### b = bert_preprocess_layer.tokenize([train_question[0]])
### bert_preprocess_layer.bert_pack_inputs([a,b])